In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","12g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .getOrCreate()

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time

# get the start time
st = time.time()
url='neo4j://localhost:7692'
password = 'password'

In [3]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from transformers import BertForTokenClassification

2023-02-19 00:53:33.970268: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 00:53:34.689424: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-19 00:53:34.689469: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-19 00:53:37.277008: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
from sparknlp.annotator import *
MODEL_NAME = '/home/worker2/Desktop/thesis/NER/NERModel_config'
bert = BertForTokenClassification.loadSavedModel(
     '{}/converting/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

2023-02-19 00:53:42.260622: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: /home/worker2/Desktop/thesis/NER/NERModel_config/converting/saved_model/1
2023-02-19 00:53:42.309064: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:55] Reading meta graph with tags { serve }
2023-02-19 00:53:42.309139: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:93] Reading SavedModel debug info (if present) from: /home/worker2/Desktop/thesis/NER/NERModel_config/converting/saved_model/1
2023-02-19 00:53:42.309248: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 00:53:42.654410: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:2

In [5]:
bert.write().overwrite().save("./{}".format(MODEL_NAME))

In [6]:
tokenClassifier_loaded = BertForTokenClassification.load("./{}".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

In [7]:
tokenClassifier_loaded.getClasses()

['I-TOOL',
 'B-TOOL',
 'I-KNOW',
 '[SEP]',
 'B-LANG',
 'I-LANG',
 'B-FRAM',
 'I-FRAM',
 'B-KNOW',
 'I-PLAT',
 '[CLS]',
 'O',
 'B-PLAT']

In [8]:
from sparknlp.base import *
document_assembler = DocumentAssembler() \
    .setInputCol('description') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

converter = NerConverter()\
    .setInputCols(["document","token","ner"])\
    .setOutputCol("ner_span")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded,
    converter
])

In [3]:
df = spark.read.option('header','true').csv('/home/worker2/Desktop/thesis/K18/File_K18/sample_crawl_dataset/Coursera_DataScience.csv',inferSchema=True, escape = '"')

In [4]:
import pyspark.sql.functions as f
df = df.withColumn("enroll", f.regexp_replace(f.col("enroll"), ",", "").alias("enroll")) \
    .withColumn("time", f.regexp_replace(f.col("time"), " hours", "").alias("time")) \
    .withColumn("time", f.regexp_replace(f.col("time"), " hour", "").alias("time")) \
    .withColumn("fee", f.regexp_replace(f.col("fee"), "[a-zA-Z]+", "").alias("fee"))
    #.withColumn("fee", f.regexp_replace(f.col("fee"), "Enroll", "").alias("fee"))

In [5]:
from pyspark.sql.types import FloatType, LongType, IntegerType

df = df \
  .withColumn("rating" ,
              df["rating"]
              .cast(FloatType()))   \
  .withColumn("enroll",
              df["enroll"]
              .cast(LongType())) \
  .withColumn("time",df['time'].cast(IntegerType())) \
  .withColumn("fee", df['fee'].cast(IntegerType()))

In [6]:
from pyspark.sql.functions import col,when
df = df.na.fill(value=0, subset=['rating','enroll', 'fee','time'])
df = df.na.fill(value='None', subset=['link','instructor', 'levelrequirement', 'skillrequirement', 'SkillWillLearn','SkillGain', 'Subject', 'organization', 'program', 'RelationInsOrg', 'Subtitle'])

In [7]:
from pyspark.sql.functions import when,lower
df = df.withColumn('levelrequirement', lower('levelrequirement'))

In [8]:
df = df.withColumn("levelrequirement", when((df.levelrequirement != "expert") & (df.levelrequirement != "advanced") & (df.levelrequirement != 'intermediate') & (df.levelrequirement != 'beginner'),"beginner") \
                                        .when(df.levelrequirement == "advanced","expert")
                                        .otherwise(df.levelrequirement))


In [9]:
from pyspark.sql.functions import when,lower
df = df.withColumn('levelrequirement', lower('levelrequirement'))

In [10]:
from pyspark.sql.functions import split, col, monotonically_increasing_id

df2 = df.select(col('name'),col('link'),col('rating'),col('enroll'),col('instructor'),col('time'),col('levelrequirement'),col('skillrequirement'),
                col('SkillWillLearn'),col('SkillGain'),col('Subject'),col('organization'),col('fee'),col('program'),col('RelationInsOrg'),split(col("Subtitle"),", ").alias("Subtitle"))
df2 = df2.withColumn("id", monotonically_increasing_id())

In [11]:
df2 = df2.withColumn("timestamp", f.current_timestamp()).alias("timestamp")

In [ ]:
regex2 = r"^https?://.*"
df2 = df2.filter(col("link").rlike(regex2))

In [12]:
course = df2.select(col('id'), col('name'), col('rating'), col('enroll'), col('time'), col('fee'), col('link'), col('timestamp'))

In [19]:
course.write.format("org.neo4j.spark.DataSource")\
    .mode("Append")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "id")\
    .option("labels",":Course")\
    .save()

In [13]:
df2 = df2.withColumn("instructor", f.regexp_replace(f.col("instructor"), ", PhD", "").alias("instructor")) \
    .withColumn("instructor", f.regexp_replace(f.col("instructor"), ", Ph.D.", "").alias("instructor")) \
    .withColumn("instructor", f.regexp_replace(f.col("instructor"), chr(34), "").alias("instructor"))

In [14]:
instructor = df2.select(col("id"), split(col("instructor"),",").alias("instructor"))

In [15]:
from pyspark.sql.functions import explode
instructor = instructor.select(instructor.id, explode("instructor").alias("instructor"))

In [16]:
instructorNode = instructor.select(instructor.instructor)

In [24]:
instructorNode.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "instructor")\
    .option("labels",":Instructor")\
    .save()

In [25]:
instructor.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'INSTRUCT_BY')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Instructor')\
    .option('relationship.target.node.keys','instructor:instructor')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [17]:
level = df2.select(col("id"), col('levelrequirement').alias('level'))

In [27]:
level.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "level")\
    .option("labels",":Level")\
    .save()

In [18]:
from pyspark.sql.functions import lower, col
subtitle = df2.select(col("id"), explode(col("Subtitle")).alias("language"))
subtitle = subtitle.withColumn('language', lower(col('language'))) \
            .withColumn("language", f.regexp_replace(f.col("language"), "subtitles: ", "").alias("language")) \
            .withColumn("language", f.regexp_replace(f.col("language"), "subtitles:", "").alias("language")) \
            .withColumn("language", f.regexp_replace(f.col("language"), "none", "english").alias("language"))

In [19]:
subNode = subtitle.select(subtitle.language)

In [30]:
subNode.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "language")\
    .option("labels",":Language")\
    .save()

In [31]:
subtitle.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'HAVE_LANGUAGE')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Language')\
    .option('relationship.target.node.keys','language:language')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [32]:
level.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'HAS_LEVEL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Level')\
    .option('relationship.target.node.keys','level:level')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [24]:
from pyspark.sql.functions import regexp_extract
regex = r"(?:https?:\/\/)?(?:[^@\n]+@)?(?:www\.)?([^:\/\n?]+)"

In [34]:
regex2 = r"^https?://.*"
website = df2.select(df2.id, df2.link)
website = website.filter(col("link").rlike(regex2))
website = website.withColumn("website", regexp_extract(df["link"], regex, 1))
websiteNode = website.dropDuplicates(['website']).select(col('website').alias('name'))

In [36]:
websiteNode.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Website")\
    .save()

In [38]:
website.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'PROVIDE_BY')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Website')\
    .option('relationship.target.node.keys','website:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [34]:
from pyspark.sql.functions import lower, col
from pyspark.sql.functions import trim,ltrim,rtrim
import pyspark.sql.functions as f
df = df.select(df.SkillWillLearn.alias('description'))
df = df.withColumn('description', lower(col('description'))) \
        .withColumn('description', f.regexp_replace(col("description"), "[\n\r]", "")) \
        .withColumn('description', f.regexp_replace(col("description"), "®", "")) \
        .withColumn('description', f.regexp_replace(col("description"), "'s", "")) \
        .withColumn('description', f.regexp_replace(col("description"), ";", " ")) \
        .withColumn('description', f.decode('description','UTF-8'))

In [35]:
result = pipeline.fit(df).transform(df)

In [36]:
from pyspark.sql.functions import split, col, monotonically_increasing_id
result = result.withColumn("id", monotonically_increasing_id())

In [37]:
import pyspark.sql.functions as F
result_last = result.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities"), 'id') \
.select(F.expr("entities['result']").alias("name"), 
        F.expr("entities['metadata'].entity").alias("entity"), 'id')

In [38]:
result_last = result_last.orderBy("id").where(result_last.entity != 'EP]')

In [39]:
programmingLanguage = result_last.distinct().select(result_last.name).where(result_last.entity == 'LANG')
#programmingLanguage = programmingLanguage.withColumn('chunk', f.regexp_replace(col("chunk"), "-?\\d+", ""))

In [40]:
programmingLanguage.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":ProgrammingLanguage")\
    .save()

In [41]:
tool = result_last.distinct().select(result_last.name).where(result_last.entity == 'TOOL')

In [42]:
tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Tool")\
    .save()

In [43]:
framework = result_last.distinct().select(result_last.name).where(result_last.entity == 'FRAM')

In [44]:
framework.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Framework")\
    .save()

In [45]:
platform = result_last.distinct().select(result_last.name).where(result_last.entity == 'PLAT')

In [46]:
platform.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Platform")\
    .save()

In [47]:
knowledge = result_last.distinct().select(result_last.name).where(result_last.entity == 'KNOW')

In [48]:
knowledge.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Knowledge")\
    .save()

In [49]:
course_lo = result_last.distinct().select(result_last.name, result_last.entity,result_last.id)

In [50]:
course_pl = course_lo.select(course_lo.name, course_lo.id).where(course_lo.entity == 'LANG')

In [51]:
course_pl.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'TEACH_PL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':ProgrammingLanguage')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [52]:
course_tool = course_lo.select(course_lo.name, course_lo.id).where(course_lo.entity == 'TOOL')
course_tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'TEACH_TOOL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Tool')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [53]:
course_fram = course_lo.select(course_lo.name, course_lo.id).where(course_lo.entity == 'FRAM')
course_fram.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'TEACH_FRAM')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Framework')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [54]:
course_plat = course_lo.select(course_lo.name, course_lo.id).where(course_lo.entity == 'PLAT')
course_plat.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'TEACH_PLAT')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Platform')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [55]:
course_know = course_lo.select(course_lo.name, course_lo.id).where(course_lo.entity == 'KNOW')
course_know.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option('batch.size','200') \
    .option("relationship", 'TEACH_KNOW')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Knowledge')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [57]:
require_df = df2.select(df2.skillrequirement.alias('description'))
require_df = require_df.withColumn('description', lower(col('description'))) \
        .withColumn('description', f.regexp_replace(col("description"), "[\n\r]", "")) \
        .withColumn('description', f.regexp_replace(col("description"), "®", "")) \
        .withColumn('description', f.regexp_replace(col("description"), "'s", "")) \
        .withColumn('description', f.regexp_replace(col("description"), ";", " ")) \
        .withColumn('description', f.decode('description','UTF-8'))

In [58]:
from sparknlp.base import *
document_assembler = DocumentAssembler() \
    .setInputCol('description') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

converter = NerConverter()\
    .setInputCols(["document","token","ner"])\
    .setOutputCol("ner_span")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded,
    converter
])

In [59]:
require_result = pipeline.fit(require_df).transform(require_df)

In [60]:
require_result = require_result.withColumn("id", monotonically_increasing_id())

In [61]:
import pyspark.sql.functions as F
require_last = require_result.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities"), 'id') \
.select(F.expr("entities['result']").alias("name"), 
        F.expr("entities['metadata'].entity").alias("entity"), 'id')

In [62]:
require_last = require_last.orderBy("id").where(require_last.entity != 'EP]')

In [63]:
pl_require = require_last.distinct().select(require_last.name).where(require_last.entity == 'LANG')
pl_require.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":ProgrammingLanguage")\
    .save()

In [64]:
tool_require = require_last.distinct().select(require_last.name).where(require_last.entity == 'TOOL')
tool_require.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Tool")\
    .save()

In [65]:
framework_require = require_last.distinct().select(require_last.name).where(require_last.entity == 'FRAM')
framework_require.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Framework")\
    .save()

In [66]:
platform_require = require_last.distinct().select(require_last.name).where(require_last.entity == 'PLAT')
platform_require.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Platform")\
    .save()

In [67]:
knowledge_require = require_last.distinct().select(require_last.name).where(require_last.entity == 'KNOW')
knowledge_require.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("node.keys", "name")\
    .option("labels",":Knowledge")\
    .save()

In [68]:
course_require_lo = require_last.distinct().select(require_last.name, require_last.entity,require_last.id)

In [69]:
course_require_pl = course_require_lo.select(course_require_lo.name, course_require_lo.id).where(course_require_lo.entity == 'LANG')
course_require_pl.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'REQUIRE_PL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':ProgrammingLanguage')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [70]:
course_require_tool = course_require_lo.select(course_require_lo.name, course_require_lo.id).where(course_require_lo.entity == 'TOOL')
course_require_tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'REQUIRE_TOOL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Tool')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [71]:
course_require_fram = course_require_lo.select(course_require_lo.name, course_require_lo.id).where(course_require_lo.entity == 'FRAM')
course_require_fram.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'REQUIRE_FRAM')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Framework')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [72]:
course_require_plat = course_require_lo.select(course_require_lo.name, course_require_lo.id).where(course_require_lo.entity == 'PLAT')
course_require_plat.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'REQUIRE_PLAT')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Platform')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [73]:
course_require_know = course_require_lo.select(course_require_lo.name, course_require_lo.id).where(course_require_lo.entity == 'KNOW')
course_require_know.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("relationship", 'REQUIRE_KNOW')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Knowledge')\
    .option('relationship.target.node.keys','name:name')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [74]:
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 959.5246350765228 seconds


In [75]:
spark.stop()